# NN for MIP to MTV

## Import Packages

In [1]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.optimizers import Adam
from IPython.display import Image, display
from keras.callbacks import CSVLogger
from matplotlib.pyplot import figure
from tensorflow.keras import models
from tensorflow.keras import layers
from PIL import Image as PIL_image
import matplotlib.pyplot as plt
from keras.models import Model
import keras.backend as K
from keras import Input
import tensorflow as tf
import urllib.request
from glob import glob
import pandas as pd
import numpy as np
import datetime
import imageio
import tarfile
import random
import time


## Make Results Reproducable

In [2]:
np.random.seed(0)
random.seed(0)
tf.random.set_seed(0)
tf.random.set_seed(0)
tf.keras.utils.set_random_seed(0)   
tf.config.experimental.enable_op_determinism()

## Variables

In [3]:
BATCH, HEIGHT, WIDTH, CHANNELS = 16, 1024, 1024, 1

In [4]:
DATASET_PATH = "dataset/"

In [5]:
RANDOM_SEED = 42

In [6]:
MODEL_NAME = "0_0-Dropout"

In [7]:
MODEL_FILENAME = "model-"+datetime.datetime.fromtimestamp(time.time()).strftime('%Y-%m-%d %H-%M-%S')

In [8]:
DROPOUT_RATE = 0.0

In [9]:
SHIFT_RANGE = 0.1

In [10]:
RANDOM_SHIFT = True

## Generator

In [11]:
class Generator(tf.keras.utils.Sequence):

    def __init__(self, df, X_col, y_col, batch_size, input_size=(HEIGHT, WIDTH, CHANNELS), shuffle=True, shift=True):
        self.df = df.copy()
        self.X_col = X_col
        self.y_col = y_col
        self.batch_size = batch_size
        self.input_size = input_size
        self.shuffle = shuffle
        self.shift = shift

        self.n = len(self.df)
        self.mtv = df[y_col['mtv']].nunique()

    def on_epoch_end(self):
        if self.shuffle:
            self.df = self.df.sample(frac=1).reset_index(drop=True)

    def __get_input(self, filepath, target_size):
        image = tf.keras.preprocessing.image.load_img(
            filepath[0], color_mode="grayscale")
        image_arr = tf.keras.preprocessing.image.img_to_array(image)
        if self.shift:
            image_arr = tf.keras.preprocessing.image.random_shift(
                image_arr, SHIFT_RANGE, SHIFT_RANGE)
        return image_arr

    def __get_output(self, label, num_classes):
        return label
        # return tf.keras.utils.to_categorical(label, num_classes=num_classes)

    def __get_data(self, batches):
        # Generates data containing batch_size samples
        filepath_batch = batches[self.X_col['filepath']]
        mtv_batch = batches[self.y_col['mtv']]
        X_batch = np.asarray([self.__get_input(x, self.input_size)
                             for x in zip(filepath_batch)])
        y0_batch = np.asarray([self.__get_output(y, self.mtv)
                              for y in mtv_batch])

        return X_batch, y0_batch

    def __getitem__(self, index):

        batches = self.df[index *
                          self.batch_size:(index + 1) * self.batch_size]
        X, y = self.__get_data(batches)
        return X, y

    def __len__(self):
        return self.n // self.batch_size


## Create Dataframes for Training/Test/Validation

##### Load Dataframe

In [12]:
train_df = pd.read_csv("train.csv", delimiter=";")
validation_df = pd.read_csv("validate.csv", delimiter=";")
test_df = pd.read_csv("test.csv", delimiter=";")


##### Use float as datatype for mtv

In [13]:
train_df['mtv'] = train_df['mtv'].astype('float64')
validation_df['mtv'] = validation_df['mtv'].astype('float64')
test_df['mtv'] = test_df['mtv'].astype('float64')

## Create Generators

In [14]:
train_generator = Generator(train_df, X_col={'filepath': 'filepath'}, y_col={
                            'mtv': 'mtv'}, batch_size=BATCH, input_size=(HEIGHT, WIDTH), shift=RANDOM_SHIFT)
test_generator = Generator(test_df, X_col={'filepath': 'filepath'}, y_col={
                           'mtv': 'mtv'}, batch_size=BATCH, input_size=(HEIGHT, WIDTH))
validation_generator = Generator(validation_df, X_col={'filepath': 'filepath'}, y_col={
                                 'mtv': 'mtv'}, batch_size=BATCH, input_size=(HEIGHT, WIDTH))


## Build Model

In [15]:
def convBlock(x, filters, kernel_size=(3, 3), dropout_rate=0.0):
    x = layers.Conv2D(filters=filters, kernel_size=kernel_size, padding="same")(x)
    x = layers.BatchNormalization()(x)
    x = layers.Activation(tf.keras.activations.relu)(x)
    x = layers.MaxPooling2D()(x)
    x = layers.Dropout(dropout_rate)(x)
    return x


In [16]:
inputs = layers.Input(shape=(HEIGHT, WIDTH, CHANNELS))
x = convBlock(inputs, 16, dropout_rate=DROPOUT_RATE)
x = convBlock(x, 32, dropout_rate=DROPOUT_RATE)
x = convBlock(x, 64, dropout_rate=DROPOUT_RATE)
x = convBlock(x, 128, dropout_rate=DROPOUT_RATE)
x = convBlock(x, 256, dropout_rate=DROPOUT_RATE)
x = convBlock(x, 512, dropout_rate=DROPOUT_RATE)
x = convBlock(x, 1024, dropout_rate=DROPOUT_RATE)
x = convBlock(x, 1024, dropout_rate=DROPOUT_RATE)
x = convBlock(x, 1024, dropout_rate=DROPOUT_RATE)
x = convBlock(x, 1024, dropout_rate=DROPOUT_RATE)

x = layers.Flatten()(x)
x = layers.Dense(1024, activation='relu')(x)
outputs = layers.Dense(1, activation='linear')(x)

model = Model(inputs=inputs, outputs=outputs)
model.compile(loss="mse", optimizer="adam", metrics=['mae'])


2022-10-02 13:04:37.640858: I tensorflow/core/platform/cpu_feature_guard.cc:151] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2022-10-02 13:04:38.063859: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1525] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 46720 MB memory:  -> device: 0, name: NVIDIA RTX A6000, pci bus id: 0000:68:00.0, compute capability: 8.6


## Show Model

In [17]:
model.summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 1024, 1024, 1)]   0         
                                                                 
 conv2d (Conv2D)             (None, 1024, 1024, 16)    160       
                                                                 
 batch_normalization (BatchN  (None, 1024, 1024, 16)   64        
 ormalization)                                                   
                                                                 
 activation (Activation)     (None, 1024, 1024, 16)    0         
                                                                 
 max_pooling2d (MaxPooling2D  (None, 512, 512, 16)     0         
 )                                                               
                                                                 
 dropout (Dropout)           (None, 512, 512, 16)      0     

## Create Callbacks

In [18]:
early_stopping_callback = tf.keras.callbacks.EarlyStopping(patience=100, monitor="val_loss", min_delta=0.1, restore_best_weights=True)

In [19]:
csv_logger = CSVLogger(MODEL_FILENAME+"_log.csv", append=False)

## Train Model

In [ ]:
history = model.fit(train_generator, epochs=1000, verbose=1, callbacks=[early_stopping_callback, csv_logger], validation_data=validation_generator)

Epoch 1/1000


2022-10-02 13:04:42.746245: I tensorflow/stream_executor/cuda/cuda_dnn.cc:368] Loaded cuDNN version 8500
2022-10-02 13:04:43.157239: I tensorflow/core/platform/default/subprocess.cc:304] Start cannot spawn child process: No such file or directory
2022-10-02 13:04:43.758455: I tensorflow/stream_executor/cuda/cuda_blas.cc:1786] TensorFloat-32 will be used for the matrix multiplication. This will only be logged once.


29/29 [==============================] - 49s 2s/step - loss: 85811.8828 - mae: 136.3233 - val_loss: 18203248640.0000 - val_mae: 134918.6406
Epoch 2/1000
 9/29 [========>.....................] - ETA: 24s - loss: 90593.0703 - mae: 136.9223

## Save Model

In [ ]:
model.save(MODEL_FILENAME)

## Plot History

In [ ]:
plt.rcParams.update({'font.size': 10})
plt.plot(history.history['mae'])
plt.plot(history.history['val_mae'])
plt.title('model mse')
plt.ylabel('mse')
plt.xlabel('epoch')
plt.legend(['train', 'val'], loc='upper left')
plt.show()